In [21]:
from pathlib import Path
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import torch as th
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy

from gc_ope.env.get_env import get_env
from gc_ope.utils.load_config_with_hydra import load_config

PROJECT_ROOT_DIR = Path().absolute().parent.parent.parent.parent
PROJECT_ROOT_DIR

PosixPath('/home/gxd/code/gc_ope')

In [8]:
# 准备env
cfg = load_config(
    config_path="../../../configs/train",
    config_name="config",
)

cfg.env.env_id = "FlyCraft-v0"

env = get_env(cfg.env)

load config from: /home/gxd/code/gc_ope/configs/env_configs/flycraft/env_config_for_ppo_easy.json
3 Generator(PCG64) Generator(PCG64)


In [10]:
# 加载policy
ckpt_path_1 = "checkpoints/flycraft/test_sac/rl_model_100000_steps.zip"
ckpt_path_2 = "checkpoints/flycraft/test_sac/rl_model_200000_steps.zip"

algo_1 = SAC.load(PROJECT_ROOT_DIR / ckpt_path_1)
algo_2 = SAC.load(PROJECT_ROOT_DIR / ckpt_path_2)

In [ ]:
# 存数据的思路一：
# 采数据的第一步：采集{(s, a, s', r, done)}

sampled_data_dict = {
    "observation": [],
    "action": [],
    "reward": [],
    "done": [],
}

# 枚举任务
desired_goal_num = 10

for _ in range(desired_goal_num):
    # 为环境设置任务
    obs, info = env.reset()
    terminate, truncated = False, False

    while not (terminate or truncated):
        action, _ = algo_2.predict(observation=obs, deterministic=True)
        
        sampled_data_dict["observation"].append(deepcopy(obs))
        sampled_data_dict["action"].append(deepcopy(action))

        obs, reward, terminate, truncated, info = env.step(action=action)

        sampled_data_dict["reward"].append(reward)
        sampled_data_dict["done"].append(terminate or truncated)

print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 279。target: (198.72, 3.20, 13.25)。achieved target: (250.40, -5.24, 24.36)。expert steps: 0。
print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 330。target: (151.23, 6.22, -3.95)。achieved target: (250.79, -10.11, 21.90)。expert steps: 0。
print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 357。target: (193.58, 1.25, -13.19)。achieved target: (258.92, 8.00, 9.08)。expert steps: 0。
print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 265。target: (238.67, -5.76, 2.56)。achieved target: (262.41, 10.22, 7.48)。expert steps: 0。
print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 144。target: (166.80, 2.32, 24.11)。achieved target: (223.83, 18.44, 30.15)。expert steps: 0。
print, Train, continuousely_move_away_termination_based_on_mu_error_and_chi_error。 steps: 250。ta

In [ ]:
# 存数据的思路二：
from stable_baselines3.common.buffers import ReplayBuffer

replay_buffer = ReplayBuffer(...)

replay_buffer.add()

replay_buffer

In [17]:
len(sampled_data_dict["observation"])

2828

In [ ]:
# TODO: 拟合$\hat_{Q}(s,a)$


In [ ]:
# stable baselines3 把dict类型的obs flatten（torch.cat），送进神经网络

tmp = {}

obs = sampled_data_dict["observation"][0]
act = sampled_data_dict["action"][0]
print(obs, act)

for ky, value in obs.items():
    tmp[ky] = th.as_tensor(obs[ky]).unsqueeze(0)

act = th.as_tensor(act).unsqueeze(0)

print(tmp, act)

q, q_target = algo_2.critic(tmp, th.as_tensor(act))
q

{'observation': array([0.5      , 0.5145179, 0.5      , 0.2      , 0.5      , 0.5      ,
       0.5      , 0.25     ], dtype=float32), 'desired_goal': array([0.19871639, 0.5177834 , 0.5367954 ], dtype=float32), 'achieved_goal': array([0.2, 0.5, 0.5], dtype=float32)} [0.9677954  0.98685855 0.92168725]
{'observation': tensor([[0.5000, 0.5145, 0.5000, 0.2000, 0.5000, 0.5000, 0.5000, 0.2500]]), 'desired_goal': tensor([[0.1987, 0.5178, 0.5368]]), 'achieved_goal': tensor([[0.2000, 0.5000, 0.5000]])} tensor([[0.9678, 0.9869, 0.9217]])


tensor([[-60.0434]], grad_fn=<AddmmBackward0>)

In [ ]:
# TODO: OPE
